# 01: General `lumispy` workflow

## Energy conversion and model fitting

This notebook shows:

- Plotting cathodoluminescence data in different ways (and interactively)
- Converting wavelength signal axis to energy signal axis
(**Required versions: hyperspy>=1.7.0 and lumispy>=0.2**)
- Gaussian fitting

**Note for M&M 2022**: This notebook was copied from a the LumiSpy demo repostitory and only minor changes were made to ensure it runs with the current Hyperspy-bundle installation. The original copy can be found at https://github.com/LumiSpy/lumispy-demos

- Joshua Taillon, July 25, 2022

Import packages:

In [ ]:
%matplotlib notebook
# Use '%matplotlib widget' in JupterLab and '%matplotlib notebook' in JupyterNotebook for interactive inline functionality (e.g. on binder)
#For pop-up window plots on your local computer, use '%matplotlib tk' or '%matplotlib qt' instead
import lumispy as lum
import hyperspy.api as hs
import os
import matplotlib.pyplot as plt

## Loading the pre-processed CL-SEM file

Load the `.hspy` file, which has already been pre-processed (background subtracted, spike removed and spectral corrected). See microscope-specific tutorials on how to do that.

*You can also leave the path empty. A pop-up window will appear to select the `.hspy` file from the browser.*

In [ ]:
# Load file
path = os.path.relpath("demo-files/01/01_demo.hspy")
cl_sem = hs.load(____, signal_type='____')

# examine the CL-SEM signal (unique to LumiSpy)
print(____)

In [ ]:
# examine the CL-SEM signal metadata
cl_sem.____

## Plotting data

Plot the hyperspectral data:

In [ ]:
# plot the hyperspectral CL-SEM data
cl_sem.____()

Plot the average CL spectrum:

In [ ]:
# Plot the mean CL spectrum
cl_sem.____().____()

### Chromatic imaging:

Plot the panchromatic image:

In [ ]:
# plot a panchromatic image
cl_sem.____.____().____(cmap='viridis')

Get the colour filtered panchromatic images.
Select the energy region to plot as coloured image interactively.

In [ ]:
# transpose the CL-SEM data
im = cl_sem.____

# plot as an image
im.____()

# create a SpanROI and interactively select the energy range to plot
roi1 = hs.roi.____(left=455, right=485) #sets a digitalbandfilter
im_roi1 = roi1.____(im, color="red")
im_roi1_mean = hs.____(im_roi1.mean,
                              event=roi1.events.changed,
                              recompute_out_event=None)

# plot the mean of the ROI
im_roi1_mean.____(cmap='viridis')

In [ ]:
# Take the mean of the energy-filtered image
im_filtered = roi1(____).____()

# calculate the width and center of the ROI
roi_width = roi1.right - roi1.left
roi_centre = roi1.left + 0.5* roi_width

# plot the filtered image with proper title
im_filtered.metadata.General.title = "Colour filtered image of {:.0f} $\pm$ {:.0f} nm".format(roi_centre, roi_width)
im_filtered.____(cmap='viridis')

## Wavelength to energy conversion

Required versions: hyperspy>=1.7.0 and lumispy>=0.2

In [ ]:
# convert CL-SEM data to eV rather than wavelength and plot
cl_sem_eV = cl_sem.____(inplace=False)
cl_sem_eV.____()

## Fitting Gaussian

Select the starting parameters

In [ ]:
# plot the data to start the fitting procedure
cl_sem_eV.____()

In [ ]:
# we recommend using the following starting parameters for the model

####################################
#MODEL
g1_centre = 2.4   # Guess for centre wavelength
g1_max = g1_centre + 0.2      # Max value for centre wavelength
g1_min = g1_centre - 0.2     # Min value for centre wavelength

g1_fwhm = 0.1            #Guess for FWHM
g1_fwhm_max = 0.5       #Maxvalue for FWHM
g1_fwhm_min = 0.01         #Minvalue for FWHM

g1_h = 5           #Guess for peak Intensity
g1_h_max = 50      #Maxvalue for peak Intesity
g1_h_min = 0         #Minvalue for peak Intensity

bkg_offset = 1  #Background to be substracted

In [ ]:
# create the CL model
m = cl_sem_eV.____()

# add a Background offset
bkg = hs.model.____.____()

# add a Gaussian peak
g1 = hs.model.____.____(
    expression="height * exp(-(x - x0) ** 2 * 4 * log(2)/ fwhm ** 2)",
    ____="Perovskite Peak", ____="x0",
    ____=1, ____=1, ____=0, ____="____")

#Tweak guessed initial parameters
m.extend([g1, bkg])

g1.x0.value, g1.x0.bmax, g1.x0.bmin = g1_centre, g1_max, g1_min
g1.fwhm.value, g1.fwhm.bmax, g1.fwhm.bmin = g1_fwhm, g1_fwhm_max, g1_fwhm_min
g1.height.value, g1.height.bmax, g1.height.bmin = g1_h, g1_h_max, g1_h_min
bkg.offset.value = bkg_offset

Fit all pixels. `iterpath` sets a continuous trail of fitting (without jumps from line to next).

In [ ]:
#Fit for all the positions
m.____(____=True, ____=True, ____='serpentine')

In [ ]:
# print model parameter values
m.____()

In [ ]:
#Plot the fit on the raw data
m.____(____=True)

In [ ]:
# Take the Gaussian component parameters as signals and plot them

# plot the x0 parameter
m_x0 = g1.____.____()
m_x0.____(cmap='inferno')

# plot the intensity
m_intensity = g1.____.____()
m_intensity.____(cmap='viridis')

### Particle segmentation
You can do particle segmentation using model fitting:

In [ ]:
#Make mask to remove region where the intensity is below the mean value:
mask_treshold = m_intensity.____.____()
mask = ____.____ > ____ #Returns a boolean matrix mask
plt.____(____)

## END
